<a href="https://colab.research.google.com/github/sh20022002/small-Language-Model/blob/main/slm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving tokenizer.zip to tokenizer (3).zip


In [2]:
import zipfile, os, sys

# Unzip the uploaded file
with zipfile.ZipFile("tokenizer.zip", 'r') as zip_ref:
    zip_ref.extractall("tokenizer")

# Add to sys.path so Python can find your modules
sys.path.append(os.path.abspath("tokenizer"))


In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset
dataset = load_dataset("tatsu-lab/alpaca")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
dataset['train']

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [6]:
# @markdown Dependency: 5Jwk6myk_LdW
from tokenizer import HybridTokenizer, Transformer, train_model
import torch

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
!ls tokenizer/tokenizer/


create_t_f.py	       __pycache__  slm.ipynb		    train.py
hybrid_tokeniztion.py  README.md    tokenizer_state.pkl.gz  transformer.py
__init__.py	       scrshots     tok_test.ipynb


In [9]:
tok = HybridTokenizer.load('tokenizer/tokenizer/tokenizer_state.pkl.gz')

In [10]:
dataset = dataset['train']

In [11]:
dataset[500]

{'instruction': 'Describe the character of Romeo from Romeo and Juliet.',
 'input': '',
 'output': 'Romeo is a tragic hero with a passionate heart. He is brave and impulsive, often making decisions without fully thinking them through. He is deeply romantic and has a heartbreaking loyalty to his family and his beloved, Juliet.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nDescribe the character of Romeo from Romeo and Juliet.\n\n### Response:\nRomeo is a tragic hero with a passionate heart. He is brave and impulsive, often making decisions without fully thinking them through. He is deeply romantic and has a heartbreaking loyalty to his family and his beloved, Juliet.'}

In [12]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset


class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Prepare raw text
        prompt = f"Q: {item['instruction']}\nA:"
        target = item['output']

        # Encode input and target using your tokenizer
        input_ids = self.tokenizer.encode(prompt, mode="flat")[:self.max_length]
        label_ids = self.tokenizer.encode(target, mode="flat")[:self.max_length]

        # Convert to tensors
        input_tensor = torch.tensor(input_ids, dtype=torch.long)
        label_tensor = torch.tensor(label_ids, dtype=torch.long)

        return {
            "input_ids": input_tensor,
            "labels": label_tensor
        }

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch, pad_id=0, ignore_index=-100):
    # 1) pad to [B, T]
    ids = [b["input_ids"] for b in batch]                                  # each [T]
    ids = pad_sequence(ids, batch_first=True, padding_value=pad_id)        # [B, T]

    # 2) build attention mask from padded ids
    attn = (ids != pad_id).long()                                          # [B, T]

    # 3) labels align 1:1 with ids; ignore pads for loss
    labels = ids.clone()
    labels[ids == pad_id] = ignore_index                                   # [B, T]

    return {"input_ids": ids, "attention_mask": attn, "labels": labels}




In [13]:
# Split the dataset using the datasets library's built-in method
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)

# Access the splits
train_data_list = split_dataset['train']
val_data_list = split_dataset['test'] # The split method names the validation split 'test' by default

# Create QADataset instances from the split data
train_dataset = QADataset(train_data_list, tok)
val_dataset = QADataset(val_data_list, tok)



In [14]:


model = Transformer(
    vocab_size=len(tok.token2id),
    dim=512,
    depth=6,
    heads=8,
    mlp_dim=2048,
    window=256
)


In [15]:

pad_id = tok.token2id["<PAD>"]      # numeric id (likely 0

# after you finish adding data to tokenizer and freeze it:
vocab = tok.vocab_size

BATCH = 8


In [16]:

from functools import partial

from torch.optim import AdamW

In [17]:


train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH,
    shuffle=True,
    collate_fn=partial(collate_fn, pad_id=pad_id, ignore_index=pad_id),
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH,
    shuffle=False,
    collate_fn=partial(collate_fn, pad_id=pad_id, ignore_index=pad_id),
)

model.resize_token_embeddings(tok.vocab_size)

assert model.token_emb.num_embeddings == vocab
assert model.to_logits.out_features   == vocab

model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)


In [18]:

# IMPORTANT: pass the SAME ignore_index to the trainer
train_model(model, train_loader, val_loader, optimizer, device, epochs=5, ignore_index=pad_id)


Epoch 1/5 - Train Loss: 8.0797, Val Loss: 7.4709, Acc: 99.14%
Epoch 2/5 - Train Loss: 6.9071, Val Loss: 6.3526, Acc: 99.69%
Epoch 3/5 - Train Loss: 5.8017, Val Loss: 5.2637, Acc: 99.87%
Epoch 4/5 - Train Loss: 4.7372, Val Loss: 4.2296, Acc: 99.94%
Epoch 5/5 - Train Loss: 3.7530, Val Loss: 3.3078, Acc: 99.96%


TypeError: 'module' object is not callable

The `TypeError` in the `train_test_split` function arises because `sklearn`'s `train_test_split` is not designed to directly handle custom PyTorch `Dataset` objects. To resolve this, we will first split the underlying data (the list of dictionaries) into training and validation sets using `train_test_split`, and then create `QADataset` instances for each split.